# Map changes: pre-fire vs. post-fire

# Install and import modules


In [2]:
pip install altair vega_datasets geopandas jsonschema==3.2 jupyter-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import altair as alt
import geopandas as gpd
from IPython.display import display, HTML

In [4]:
from google.colab import drive
drive.mount('/content/drive')
PATH = '/content/drive/MyDrive/Colab Notebooks/CSE 512/A4'

Mounted at /content/drive


# Read in data

In [50]:
# v for visits
v = pd.read_csv(f'{PATH}/monthly_vis_by_site_zeroed.csv')
v.drop(v[(v["month"]==9) & (v["year"]==2016) & (v["years_fire"]==-2)].index, inplace = True)
v.head()

,siteid,Name,month,year,years_fire,pre_post_fire,burned,closed_ever,tot_vis,closed_month,d2p
0,CRG_002,Oneonta Gorge,1,2015,-3,pre,yes,yes,169,open,2015-01-15
1,CRG_002,Oneonta Gorge,1,2016,-2,pre,yes,yes,436,open,2016-01-15
2,CRG_002,Oneonta Gorge,1,2017,-1,pre,yes,yes,788,open,2017-01-15
3,CRG_002,Oneonta Gorge,1,2018,0,post,yes,yes,0,closed,2018-01-15
4,CRG_002,Oneonta Gorge,1,2019,1,post,yes,yes,0,closed,2019-01-15


In [51]:
# mc for monthly changes
mc = pd.read_csv(f'{PATH}/monthlypred.csv')
mc.head()

,sepyr,siteid,month,Name,TAVG,PRCP,tot_vis,year,burned,closed_month,d2p,pred,pctdiff,diff
0,2015,CRG_002,1,Oneonta Gorge,41.1,7.97,169,2015,yes,open,2015-01-15,482.645657,-0.649847,-313.645657
1,2015,CRG_014,1,Tom McCall TNC trail,41.1,7.97,45,2015,no,open,2015-01-15,50.208703,-0.103741,-5.208703
2,2015,CRG_033,1,Falls Creek Falls,41.1,7.97,91,2015,no,open,2015-01-15,80.380664,0.132113,10.619336
3,2015,CRG_031,1,Eight Mile trail,41.1,7.97,28,2015,no,open,2015-01-15,11.862908,1.360298,16.137092
4,2015,CRG_029,1,Umbrella/Sahalie Falls,41.1,7.97,97,2015,no,open,2015-01-15,137.761641,-0.295885,-40.761641


In [31]:
data_geojson = gpd.read_file(f'{PATH}/EagleCreek_AOI.geojson')
data_geojson.head()

,descriptio,layer,path,Name,siteid,geometry
0,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,..."
1,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Bell Creek,CRG_003,"MULTIPOLYGON Z (((-122.06645 45.55340 0.00000,..."
2,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Horsetail Creek,CRG_004,"MULTIPOLYGON Z (((-122.00751 45.57873 0.00000,..."
3,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,"Nesmith Point, McCord, Elowa Falls",CRG_005,"MULTIPOLYGON Z (((-122.01195 45.61198 0.00000,..."
4,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Munra Falls,CRG_006,"MULTIPOLYGON Z (((-121.95856 45.63083 0.00000,..."


In [45]:
geo_mchg = data_geojson.merge(mc, on=['siteid', 'Name'])
geo_mchg.drop(geo_mchg[(geo_mchg["year"]<2017)].index, inplace = True)
geo_mchg.drop(geo_mchg[(geo_mchg["year"]==2017) & (geo_mchg["month"]<9)].index, inplace = True)
pd.crosstab(geo_mchg["year"], geo_mchg["month"])

month,1,2,3,4,5,6,7,8,9,10,11,12
year,,,,,,,,,,,,
2017,0,0,0,0,0,0,0,0,41,41,41,41
2018,41,41,41,41,41,41,41,41,41,41,41,41
2019,41,41,41,41,41,41,41,41,0,0,0,0


In [46]:
#TODO: Heed the warning about geographic vs projected CRS?
geo_vis['centroid_lon'] = geo_vis['geometry'].centroid.x
geo_vis['centroid_lat'] = geo_vis['geometry'].centroid.y
geo_vis.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  This is separate from the ipykernel package so we can avoid doing imports until


,descriptio,layer,path,Name,siteid,geometry,month,year,years_fire,pre_post_fire,...,closed_ever,tot_vis,closed_month,d2p,centroid_lon,centroid_lat,pre_fire_tot_x,pre_fire_rank_x,pre_fire_tot_y,pre_fire_rank_y
0,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2015,-3,pre,...,yes,169,open,2015-01-15,-122.071751,45.571264,173482,1,173482,1
1,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2016,-2,pre,...,yes,436,open,2016-01-15,-122.071751,45.571264,173482,1,173482,1
2,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2017,-1,pre,...,yes,788,open,2017-01-15,-122.071751,45.571264,173482,1,173482,1
3,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2018,0,post,...,yes,0,closed,2018-01-15,-122.071751,45.571264,173482,1,173482,1
4,None,Eagle_ creek_substitutes_111319 Closed,/home/sgwinder/Documents/Fires/GIS/Eagle_ cree...,Oneonta Gorge,CRG_002,"MULTIPOLYGON Z (((-122.07717 45.58856 0.00000,...",1,2019,1,post,...,yes,0,closed,2019-01-15,-122.071751,45.571264,173482,1,173482,1


## Sort/Filter based on pre-fire trail popularity

In [60]:
pre_fire_total = v[(v['pre_post_fire'] == 'pre')].groupby(['Name'], as_index=False)['tot_vis'].sum()
pre_fire_total = pre_fire_total.sort_values(by='tot_vis', ascending=False, ignore_index=True)
pre_fire_total['pre_fire_rank'] = pre_fire_total.index + 1
pre_fire_total = pre_fire_total.rename(columns={'tot_vis': 'pre_fire_tot'})
pre_fire_total

geo_mchg = geo_mchg.merge(pre_fire_total, on='Name')
geo_mchg.head()

,descriptio,layer,path,Name,siteid,geometry,sepyr,month,TAVG,PRCP,...,pre_fire_tot_x,pre_fire_rank_x,pre_fire_tot_y,pre_fire_rank_y,pre_fire_tot_x,pre_fire_rank_x,pre_fire_tot_y,pre_fire_rank_y,pre_fire_tot,pre_fire_rank


# Build charts

# Build charts

In [56]:
### SELECTIONS

# select trail
multi = alt.selection_multi(
    fields=['Name','siteid'],
    empty='none'
)

# hover nearest date
nearest = alt.selection(
        type="single", nearest=True, on="mouseover", fields=["d2p"], empty="none", init={'d2p':1410739200000}
    )

# slider to choose top N in popularity
slider = alt.binding_range(min=1, max=41, step=1, name="Top # Most Popular Trails: ")
topN = alt.selection_single(name='filter', fields=['cutoff'], bind=slider, init={'cutoff': 20})

### LINE CHART

# Draw a rule at the location of the selection
rules = alt.Chart(geo_mchg).mark_rule(
        color="black", 
        strokeWidth=1
    ).encode(
        x="d2p",
    ).transform_filter(nearest)

# Helps draw the rules, this is what we are using to hover
selectors = alt.Chart(geo_mchg).mark_point().encode(
            x="d2p",
            opacity=alt.value(0), # Make dots invisible. (Can turn opacity to 1 to see what this is doing)
        ).add_selection(nearest)

# This is the main line chart that shows total visits
line_chart = alt.Chart(geo_mchg).mark_line().encode(
                    x=alt.X('yearmonth(d2p):T',title='Date'),
                    y=alt.Y('diff:Q',title='Change in visitors relative to pre-fire'),
                    color = alt.Color('Name:N', scale=alt.Scale(range = ['gray', 'gray']), legend=None),
                    strokeWidth=alt.condition(multi, alt.value(4), alt.value(1)),
                    opacity=alt.condition(multi, alt.value(1), alt.value(0.30)),
                ).transform_filter(
                   alt.datum.pre_fire_rank <= topN.cutoff
                ).add_selection(
                    topN
                ).properties(
                    width=600,
                    height=200,
                    title={
                          "text": ["Outdoor Recreation Before and After the Eagle Creek Fire","What were trends in trail use before and after a major fire in the Columbia Gorge?"],
                          "subtitle": ["", "Pan over the line chart to visualize the difference between post-fire visits and an estimate based on pre-fire data"],
                          }
                )

# Mark's when Eagle Creek Fire happened
fire_rule = alt.Chart(geo_mchg).mark_rule(
        color="orange", 
        strokeWidth=5
    ).encode(
        x="d2p",
    ).transform_filter("datum.year == 2017 && datum.month == 9")

fire_text = alt.Chart(geo_mchg).mark_text(
    text = 'Eagle Creek Fire', 
    angle=270,
    fontSize=12
).encode(
        x="d2p",
    ).transform_filter("datum.year == 2017 && datum.month == 10")

# Denote each year
year_rule = alt.Chart(geo_mchg).mark_rule(
        color='black',
        strokeDash=[3,5]
    ).encode(
        x="d2p",
    ).transform_filter("[ datum.year == 2017 || datum.year == 2018 || datum.year == 2019] && datum.month == 1")
    

################## MAP STUFF

# name of current selected trail
trail_title_labels = alt.Chart(geo_mchg).mark_text(align='left').encode(
    x = alt.value(0),
    y = alt.value(10),
    text= 'Name',
    size=alt.value(20),
    opacity=alt.condition(multi, alt.value(1), alt.value(0))
)

# Display current date
date_label = alt.Chart(geo_mchg).mark_text(align='right').encode(
    x = alt.value(450),
    y = alt.value(10),
    text= 'yearmonth(d2p)',
    size=alt.value(15),
).transform_filter(nearest)


# 
choromap = alt.Chart(geo_mchg).mark_geoshape(
        stroke='black',
        strokeWidth=1,
        color = 'black'
    ).encode(
        #stroke=alt.Color('burned', title = 'Burned', scale=alt.Scale(range=['steelblue', 'orange'])),
        #strokeDash=alt.StrokeDash('closed_month', title = 'Open/Closed?'), 
        stroke=alt.Color('burned', 
                         title = 'Burned', 
                         scale=alt.Scale(range=['steelblue', 'orange']), 
                         legend=alt.Legend(
                            orient='bottom',
                             symbolStrokeWidth=5,
                            titleAnchor='middle')
                        ),
        strokeDash=alt.StrokeDash('closed_month', 
                                  title = 'Open/Closed?', 
                                  scale=alt.Scale(domain=['closed', 'open']),
                                        legend=alt.Legend(
                                        orient='bottom',
                                        symbolDash=[5,2],
                                         symbolStrokeColor='black',
                                         symbolFillColor='green', # this is required... but not displayed
                                         symbolType='stroke',
                                        titleAnchor='middle')
                                 ),  
        strokeWidth=alt.condition(alt.datum.burned == 'yes',alt.value(3),alt.value(2)),
        #color = 'black',
        tooltip=['Name', alt.Tooltip('diff:Q', title='Change relative to pre-fire'), alt.Tooltip('yearmonth(d2p):T', title='Date'), alt.Tooltip('closed_month', title='Open/Closed') ],
        color=alt.Color("diff:Q", title = 'Change relative to pre-fire', scale=alt.Scale(scheme="redblue", domain=[-20000, 20000]))
    ).add_selection(
        multi
    ).transform_filter(nearest).properties(
    width=600,
    height=400,
    title={
    "text": " ", 
    "subtitle": ["Hover over each trail area for more info","Click to highlight visitor trends above","Highlight multiple trails with shift+click"],
    }
).transform_filter(
    alt.datum.pre_fire_rank <= topN.cutoff
).add_selection(
    topN
)

# mark text/symbol directly on shape of currently selected trail
trail_marker = alt.Chart(geo_mchg).mark_text(baseline='top'
 ).encode(
     longitude='centroid_lon:Q',
     latitude='centroid_lat:Q',
     text='Name',
     size=alt.value(16),
     opacity=alt.condition(multi, alt.value(1), alt.value(0))
 ).properties( 
    width=600,
    height=300
)

final = (line_chart + selectors + rules + fire_rule + fire_text + year_rule) & (choromap + date_label + trail_marker)
final

alt.VConcatChart(...)